# Joins columns from benchmarks and jobsarchive tables and outputs them for ML analysis

In [1]:
import pickle
import socket
import re
import datetime
from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import scan

import numpy as np
import pandas as pd

es = Elasticsearch([{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)

#start_date = datetime.datetime(2017, 4, 7)
start_date = datetime.datetime(2017, 6, 1)
#curre_date = datetime.datetime(2017, 4, 30, 23, 59, 59)
curre_date = datetime.datetime.utcnow()
days_of_data = (curre_date - start_date).days

benchmark_indices = ['benchmarks-2017.07','benchmarks-2017.08']

In [2]:
# get job archive indices
indices = es.cat.indices(index="jobs_archive_20*", h="index", request_timeout=600).split('\n')
indices = sorted(indices)
indices = [x for x in indices if x != '']
#for i in indices:
#    print(i)

selected_indices=[] 
for d in range(days_of_data+1):
    td = start_date + datetime.timedelta(d)
    ind = 'jobs_archive_'+td.strftime('%Y-%m-%d')
    if ind in indices:
        selected_indices.append(ind)

job_indices = ','.join(selected_indices)
print(job_indices)

jobs_archive_2017-06-01,jobs_archive_2017-06-02,jobs_archive_2017-06-03,jobs_archive_2017-06-04,jobs_archive_2017-06-05,jobs_archive_2017-06-06,jobs_archive_2017-06-07,jobs_archive_2017-06-08,jobs_archive_2017-06-09,jobs_archive_2017-06-10,jobs_archive_2017-06-11,jobs_archive_2017-06-12,jobs_archive_2017-06-13,jobs_archive_2017-06-14,jobs_archive_2017-06-15,jobs_archive_2017-06-16,jobs_archive_2017-06-17,jobs_archive_2017-06-18,jobs_archive_2017-06-19,jobs_archive_2017-06-20,jobs_archive_2017-06-21,jobs_archive_2017-06-22,jobs_archive_2017-06-23,jobs_archive_2017-06-24,jobs_archive_2017-06-25,jobs_archive_2017-06-26,jobs_archive_2017-06-27,jobs_archive_2017-06-28,jobs_archive_2017-06-29,jobs_archive_2017-06-30,jobs_archive_2017-07-01,jobs_archive_2017-07-02,jobs_archive_2017-07-03,jobs_archive_2017-07-04,jobs_archive_2017-07-05,jobs_archive_2017-07-06,jobs_archive_2017-07-07,jobs_archive_2017-07-08,jobs_archive_2017-07-09,jobs_archive_2017-07-10,jobs_archive_2017-07-11,jobs_archive_201

In [3]:
benchmarks_query={
    "size": 0,
    "query":{
       "range": {
            "metadata.threadsPerCore":{
                "gte":"0"
            }
        }
    }
}

res = scan(es, query=benchmarks_query, index=benchmark_indices, scroll='5m', timeout="5m", size=1000)

data=[]
benchmarksPerSite={}
count = 0
for r in res:
    pr=r['_source']['profiles']
    mt=r['_source']['metadata']
    host = mt['node']
    if '@' in host: 
        host = host[host.index('@')+1:]
    site=mt['ATLASSite']
    if site not in benchmarksPerSite:
        benchmarksPerSite[site]=0
    if site.startswith('BOINC'): continue # for now skip all the possible laptops
    benchmarksPerSite[site]+=1
    doc=[
        mt['ip'], host, site, mt['meminfo'],
        mt['mp_num'],mt['cpuname'],mt['isAVM'],mt['cpunum'],mt['coresPerSocket'],mt['threadsPerCore']
    ]
    data.append(doc)
    if not count%10000:
        print(count)
#     if count>100: break
    count+=1

print(len(data))
    
benchmark=pd.DataFrame(data,columns=['ip','host','site','meminfo','mpnum','cpuname','VM','cpunum','coresPerSocket','threadsPerCore'])
del data

0
10000
20000
30000
33270


In [4]:
#print(benchmark.dtypes)
benchmark.head()

,ip,host,site,meminfo,mpnum,cpuname,VM,cpunum,coresPerSocket,threadsPerCore
0,192.168.25.14,r25-n14.ph.liv.ac.uk,UKI-NORTHGRID-LIV-HEP_SL6,49390764.0,1,Intel(R) Xeon(R) CPU E5-2630 v2 @ 2.60GHz,false,24,6,2
1,188.185.193.56,cern-atlas-dcb8d293-709a-42f5-b891-8f3eface243...,CERN-PROD_CLOUD_MCORE,15031436.0,8,Intel(R) Xeon(R) CPU E5-2650 v2 @ 2.60GHz,true,8,1,1
2,131.154.202.135,wn-200-03-27-01-a.cr.cnaf.infn.it,INFN-T1_MCORE,65950436.0,8,AMD Opteron(tm) Processor 6320,false,16,4,2
3,146.179.247.114,wg60.grid.hep.ph.ic.ac.uk,UKI-LT2-IC-HEP_MCORE,49279700.0,8,Intel(R) Xeon(R) CPU X5650 @ 2.67GHz,false,24,6,2
4,169.254.169.3,vac037-03.beowulf.cluster,UKI-SCOTGRID-GLASGOW_MCOREVAC,16428256.0,1,QEMU Virtual CPU version 1.5.3,true,8,1,1


In [5]:
benchmark.drop_duplicates(subset='host',inplace=True)

In [6]:
#IPs=set(benchmark['ip'].tolist())
HOSTs=set(benchmark['host'].tolist())

# print('total IPs:', len(IPs))
# for ip in IPs:
#     try:
#         res=socket.gethostbyaddr(ip)
#         h=res[0]
#         HOSTs.add(h)
#         print(res)
#     except:
#         pass
print('total HOSTs:', len(HOSTs))

total HOSTs: 14071


In [7]:
benchmark.to_csv('benchmark_node_info.csv',index=False)


In [8]:
# select only production jobs as we are currently only interested in these. 
# prodsourcelabel = prod_test, rc_test, managed, install
job_query = {
    "size": 0,
    "_source": [ 
        "modificationhost", "cpuconsumptiontime", "wall_time","nevents","inputfilebytes",
        "prodsourcelabel","processingtype","computingsite","jeditaskid"
    ],
    'query':{
            'bool':{
                   'must':[
                        { "term": {"jobstatus": "finished" } },
                        { "term": {"prodsourcelabel": "managed" } }
                    ],
                   'should':[
                      # { "term": {"prodsourcelabel": "rc_test" } },
                      # { "term": {"prodsourcelabel": "prod_test" } },
                       { "term": {"prodsourcelabel": "managed" } }
                   ]
            }
    }
                
}

data=[]
scroll = scan(client=es, index=job_indices, query=job_query, scroll='5m', timeout="5m", size=10000)
count = 0
counttype={'managed':0,'rc_test':0,'prod_test':0,'install':0}
jobsPerSite={}
for res in scroll:
    count += 1
    #print(res)
    #if count>3: break
    
    r=res['_source']
    
    site=r['computingsite']
    if site not in jobsPerSite:
        jobsPerSite[site]=0
    jobsPerSite[site]+=1
    
    if not count%100000: 
        print(count, ' selected:', counttype)
        #print(data)
    
    
    host=r['modificationhost']
    if host is None: continue
        
    if '@' in host: 
        host = host[host.index('@')+1:]
    #if '.' in host: host=host[:host.index('.')]   
    if host not in HOSTs: continue
    
    counttype[r['prodsourcelabel']]+=1
    cpu=r['cpuconsumptiontime']
    ifb=r['inputfilebytes']
    wall=r['wall_time']
    nevents=r['nevents']
    if wall>0: cpueff = cpu/wall
    if nevents>0:
        wallPerEvent = wall/nevents
        cpuPerEvent = cpu/nevents
    else:
        wallPerEvent = 0
        cpuPerEvent =0
    doc=[host,cpu,wall,r['processingtype'],nevents, cpueff, wallPerEvent, cpuPerEvent,ifb,r['jeditaskid']]
    data.append(doc)

job=pd.DataFrame(data,columns=['host','cputime','walltime','processingtype','nevents','cpueff', 'wallPerEvent', 'cpuPerEvent','inputsize','taskid'])
del data

100000  selected: {'prod_test': 0, 'managed': 56515, 'install': 0, 'rc_test': 0}
200000  selected: {'prod_test': 0, 'managed': 114815, 'install': 0, 'rc_test': 0}
300000  selected: {'prod_test': 0, 'managed': 171956, 'install': 0, 'rc_test': 0}
400000  selected: {'prod_test': 0, 'managed': 227701, 'install': 0, 'rc_test': 0}
500000  selected: {'prod_test': 0, 'managed': 284551, 'install': 0, 'rc_test': 0}
600000  selected: {'prod_test': 0, 'managed': 341734, 'install': 0, 'rc_test': 0}
700000  selected: {'prod_test': 0, 'managed': 399293, 'install': 0, 'rc_test': 0}
800000  selected: {'prod_test': 0, 'managed': 456838, 'install': 0, 'rc_test': 0}
900000  selected: {'prod_test': 0, 'managed': 513052, 'install': 0, 'rc_test': 0}
1000000  selected: {'prod_test': 0, 'managed': 568914, 'install': 0, 'rc_test': 0}
1100000  selected: {'prod_test': 0, 'managed': 625102, 'install': 0, 'rc_test': 0}
1200000  selected: {'prod_test': 0, 'managed': 681991, 'install': 0, 'rc_test': 0}
1300000  selec

In [9]:
job.head()

,host,cputime,walltime,processingtype,nevents,cpueff,wallPerEvent,cpuPerEvent,inputsize,taskid
0,ba-3-9-17.cr.cnaf.infn.it,144004,155257,evgen,50,0.927520,3105.140000,2880.080000,0,11330815
1,td113.pic.es,21474,3353,recon,250,6.404414,13.412000,85.896000,1027539018,11470165
2,b63ab874b9.cern.ch,45890,46366,simul,1000,0.989734,46.366000,45.890000,211808076,11374294
3,t2-wn-91.mi.infn.it,2492,4207,pmerge,3878,0.592346,1.084838,0.642599,789365504,11570096
4,c-104-15.aglt2.org,3712,5117,recon,17,0.725425,301.000000,218.352941,24052972,11518326


#### store all the jobs together

In [10]:
job.to_csv('job_node_info.csv',index=False)

#### store jobs split per processingtype

In [11]:
UniqueProcessingTypes = job.processingtype.unique()
print(UniqueProcessingTypes)

#create a dict for all the dataframes to be filled later
ProcessingType = {elem : pd.DataFrame for elem in UniqueProcessingTypes}

#filling up data frames
for key in ProcessingType.keys():
    job[:][job.processingtype == key].to_csv(key+'_node_info.csv',index=False)

['evgen' 'recon' 'simul' 'pmerge' 'deriv' 'merge' 'pile' 'reprocessing'
 'overlay' 'eventIndex' 'digit']


In [12]:
#full = pd.merge(benchmark, job, on='pandaid')
#full.head()
#full.to_csv('benchmark_job.csv',index=False)

In [13]:
#for ps in benchmarksPerSite:
#    print(ps, benchmarksPerSite[ps])

for ps in jobsPerSite:
    if ps not in benchmarksPerSite and jobsPerSite[ps]>1000: 
        print(ps, jobsPerSite[ps])#,'       <<<<<<<<<<<<<')
    #else:
        #print(ps, jobsPerSite[ps],benchmarksPerSite[ps])

CERN-P1_DYNAMIC_MCORE 18715
UKI-NORTHGRID-LANCS-HEP_MCORE 29609
UKI-NORTHGRID-LANCS-HEP_SL6 102441
CONNECT_JETSTREAM 14203
FZK-LCG2_MCORE_HI 3556
CERN-P1_DYNAMIC_SCORE 16010
CONNECT_ICCHEPT3 2780
SFU-LCG2 17429
UKI-LT2-IC-HEP_SL7 46859
ORNL_Titan_MCORE 1284802
CONNECT_ES_ICC 2982
UKI-LT2-IC-HEP_MCORE_SL7 4391
BNL_ATLAS_2 3272
RAL-AZURE_VAC 4268
SiGNET 32948
UKI-NORTHGRID-LANCS-HEP_ES 1297
CONNECT_ES_ODYSSEY 25386
OU_OSCER_ATLAS_MCORE 11191
LRZ-LMU_MCORE_LO 31404
BNL_PROD 494781
LUNARC_MCORE 2925
CERN-P1_DYNAMIC_MCORE_LOWMEM 46854
CONNECT_ES_KARST 1714
BNL_PROD_CONTR 107682
OU_OSCER_ATLAS_OPP 19604
CONNECT_ES_ODYSSEY_MCORE 6978
CONNECT_FRESNOSTATE 3137
LRZ-LMU_MUC1_MCORE 136427
BNL_PROD_MCORE 577935
CONNECT_KARST 6107
CONNECT_ES_JETSTREAM 1210
SiGNET-NSC_MCORE 14199
CONNECT_UIUC 2369
BNL_LOCAL 1491
CONNECT_ES_ICC_MCORE 1300
SFU-LCG2_MCORE 13581
CA-VICTORIA-WESTGRID-T2 35439
Anselm_MCORE 4160
CONNECT_JETSTREAM_MCORE 1542
SiGNET_MCORE 91672
CPPM_MCORE 19044
OU_OSCER_ATLAS 19206
CERN-PROD_

In [14]:
print (jobsPerSite)
#print (benchmarksPerSite)

{'RAL-LCG2_SL6': 43961, 'UTA_SWT2': 20746, 'TRIUMF_MCORE_LOMEM': 234372, 'CYFRONET-LCG2': 1190, 'TOKYO_MCORE_ARC': 100694, 'INFN-T1-BA': 151613, 'IN2P3-CC_HIMEM': 116014, 'TW-FTT_SL6': 37424, 'UKI-NORTHGRID-MAN-HEP_SL6': 179821, 'UIO_MCORE_LOPRI': 9390, 'CERN-PROD_PUB_SCORE_SHORT': 337342, 'UNI-FREIBURG': 57347, 'IN2P3-CC': 141374, 'RAL-LCG2_VHIMEM': 9522, 'IFIC': 64949, 'UIO_MCORE': 30733, 'IL-TAU-HEP': 14549, 'GoeGrid': 210131, 'ATLAS_OPP_OSG-FNAL_GPCE01': 400, 'CERN-P1_DYNAMIC_MCORE': 18715, 'HU_ATLAS_Tier2_MCORE': 31847, 'INFN-COSENZA-RECAS_MCORE': 53850, 'UKI-NORTHGRID-LANCS-HEP_MCORE': 29609, 'GRIF-LPNHE_MCORE': 41482, 'INFN-BOLOGNA-T3': 14009, 'UKI-NORTHGRID-LANCS-HEP_SL6': 102441, 'FMPhI-UNIBA_MCORE': 5319, 'INFN-ROMA3_MCORE': 1211, 'BU_ATLAS_Tier2_MCORE': 127847, 'CONNECT_JETSTREAM': 14203, 'UKI-SCOTGRID-GLASGOW_MCORE': 88020, 'CERN-PROD-preprod_MCORE': 184134, 'INFN-MILANO-ATLASC': 102474, 'DESY-ZN': 59445, 'INFN-NAPOLI-ATLAS_MCORE': 29178, 'CERN-PROD_T0_4MCORE': 235379, 'SAR